In [3]:
import sys
!{sys.executable} -m pip install nba_api

                                              0.0/257.3 kB ? eta -:--:--
     -------------------------------        215.0/257.3 kB 6.6 MB/s eta 0:00:01
     ----------------------------------     235.5/257.3 kB 4.8 MB/s eta 0:00:01
     ----------------------------------     235.5/257.3 kB 4.8 MB/s eta 0:00:01
     ----------------------------------     235.5/257.3 kB 4.8 MB/s eta 0:00:01
     ----------------------------------     235.5/257.3 kB 4.8 MB/s eta 0:00:01
     ----------------------------------     235.5/257.3 kB 4.8 MB/s eta 0:00:01
     ----------------------------------     235.5/257.3 kB 4.8 MB/s eta 0:00:01
     -----------------------------------  256.0/257.3 kB 655.4 kB/s eta 0:00:01
     ------------------------------------ 257.3/257.3 kB 608.4 kB/s eta 0:00:00
                                              0.0/62.6 kB ? eta -:--:--
     ---------------------------------------- 62.6/62.6 kB 3.3 MB/s eta 0:00:00
                                              0.0/96.6 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from nba_api.stats.static import teams,players
from nba_api.stats.endpoints import playergamelog, playerindex
import requests
import pandas as pd
import time
import os

In [2]:
# Retry Wrapper 
def retry(func, retries=3):
    def retry_wrapper(*args, **kwargs):
        attempts = 0
        while attempts < retries:
            try:
                return func(*args, **kwargs)
            except requests.exceptions.RequestException as e:
                print(e,attempts)
                time.sleep(30)
                attempts += 1

    return retry_wrapper


@retry
def getAggPlayerStats(pid,fname,lname,position,df,minGames=50):
        
    #get player game logs    
    data = playergamelog.PlayerGameLog(player_id=pid,season='2022-23')
    data = data.player_game_log.get_data_frame()

    #remove less than n games
    if len(data) < minGames: return pd.DataFrame()
    
    # build row for player stats
    data = data[['MIN','FG_PCT','FT_PCT','FG3M','REB','AST','STL','BLK','PTS']]
    mean_df  = data.mean().to_frame().T
    mean_df.columns = [col + '_MEAN' for col in mean_df.columns]
    std_df = data.std().to_frame().T
    std_df.columns = [col + '_STD' for col in std_df.columns]

    # Join all into one table
    data = pd.concat([mean_df,std_df],axis=1)
    data['player_first_name'] = fname
    data['player_last_name'] = lname
    data['player_id'] = pid
    data['position'] = position

    #concat table to bigger df
    final_df = pd.concat([df,data])
    return final_df
        


         




In [3]:
df = pd.DataFrame()
players = playerindex.PlayerIndex().player_index.get_data_frame()
if os.path.exists('fantasy_stats.csv'):
    df = pd.read_csv('fantasy_stats.csv').drop('Unnamed: 0',axis=1)
    players = players[~players['PERSON_ID'].isin(df['player_id'])]

for _,player in players.iterrows():
    data = getAggPlayerStats(player['PERSON_ID'],player['PLAYER_FIRST_NAME'],player['PLAYER_LAST_NAME'],player['POSITION'],df)
    if len(data) > 0:
        df = data

df.to_csv('fantasy_stats.csv')

In [4]:
df


,MIN_MEAN,FG_PCT_MEAN,FT_PCT_MEAN,FG3M_MEAN,REB_MEAN,AST_MEAN,STL_MEAN,BLK_MEAN,PTS_MEAN,MIN_STD,...,FG3M_STD,REB_STD,AST_STD,STL_STD,BLK_STD,PTS_STD,player_first_name,player_last_name,player_id,position
0,20.781818,0.459564,0.430309,0.527273,5.963636,0.909091,0.563636,0.545455,9.236364,9.007591,...,0.766293,4.104937,1.110101,0.995613,0.689019,5.517526,Precious,Achiuwa,1630173,F
1,34.640000,0.542920,0.730613,0.013333,9.173333,3.200000,1.173333,0.813333,20.386667,4.738115,...,0.115470,3.422396,1.692591,1.031574,0.910796,6.610584,Bam,Adebayo,1628389,C-F
2,20.491525,0.378661,0.294915,1.372881,2.050847,1.135593,0.271186,0.254237,7.915254,10.756510,...,1.460967,1.591180,1.431874,0.485319,0.632271,7.052239,Ochai,Agbaji,1630534,G
3,21.870130,0.467039,0.414026,1.220779,4.818182,1.259740,0.584416,0.623377,9.038961,6.081358,...,1.059088,2.817835,1.068725,0.783895,0.811739,5.115615,Santi,Aldama,1630583,F-C
4,14.949153,0.399102,0.258186,1.033898,1.711864,1.830508,0.542373,0.355932,6.152542,8.484110,...,1.272604,1.661437,1.620478,0.915801,0.549693,5.641024,Nickeil,Alexander-Walker,1629638,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,25.955224,0.521224,0.673060,1.567164,7.328358,1.805970,0.447761,1.074627,16.626866,7.510463,...,1.305279,3.417371,1.734139,0.839978,1.209977,7.244299,Christian,Wood,1626174,F
294,24.500000,0.457360,0.453340,0.820000,3.600000,3.880000,1.840000,0.340000,7.380000,5.990639,...,1.100835,2.128523,2.487848,1.489555,0.626295,4.989336,Delon,Wright,1626153,G
295,14.722222,0.486222,0.143519,0.111111,3.074074,1.388889,1.000000,0.092593,4.444444,8.710039,...,0.371974,2.561337,1.337650,1.197482,0.292582,4.068594,Thaddeus,Young,201152,F
296,34.835616,0.427877,0.860192,2.109589,2.972603,10.150685,1.095890,0.123288,26.219178,3.851456,...,1.532623,1.787088,3.156436,1.132406,0.331042,7.414787,Trae,Young,1629027,G
